In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb


In [2]:
run = wandb.init(project="deep-origin-task", name="test")

wandb: Currently logged in as: stepan-v-kuznetsov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [5]:
run.log({'loss': 1})

In [6]:
seqs = pd.read_parquet('../pipeline/output/train_data/train_filtered.parquet')

seqs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25012 entries, 0 to 25011
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               25012 non-null  object 
 1   pdb_id           25012 non-null  object 
 2   chain_id         25012 non-null  object 
 3   length           25012 non-null  int64  
 4   sequence         25012 non-null  object 
 5   residue_indices  25012 non-null  object 
 6   b_factors        25012 non-null  object 
 7   mean_b_factor    25012 non-null  float64
 8   coords_ca        25012 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.7+ MB


In [7]:
seqs.iloc[0]

id                                                      train_106M_A
pdb_id                                                          106M
chain_id                                                           A
length                                                           154
sequence           MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
residue_indices    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
b_factors          [32.86, 27.2, 20.55, 18.05, 19.76, 15.24, 11.0...
mean_b_factor                                              16.246299
coords_ca          [24.708999633789062, 9.586000442504883, -9.829...
Name: 0, dtype: object

In [20]:
import torch

batch_size = 2
num_layers = 2
num_heads = 3
seq_len = 2

# Создаём список attention maps, по одному для каждого слоя
attentions = []
for layer in range(num_layers):
    layer_heads = []
    for head in range(num_heads):
        value = layer*num_heads + head+1  # уникальное значение для каждого слоя/головы
        # attention shape: (batch_size, seq_len, seq_len)
        att = torch.full((seq_len, seq_len), value)
        layer_heads.append(att)
    # Собираем в один слой: (num_heads, seq_len, seq_len)
    layer_tensor = torch.stack(layer_heads, dim=0)
    batch_tensor = torch.stack([layer_tensor*(-1), layer_tensor], dim=0)
    attentions.append(batch_tensor)

# Stack по слоям: (num_layers, batch_size, num_heads, seq_len, seq_len)
all_attentions = torch.stack(attentions)
print("all_attentions shape:", all_attentions.shape)
print(all_attentions)

for batch in range(batch_size):
    print(f"Batch {batch} before view")
    print(all_attentions[:, batch, :, :, :] )

# Переформатируем:
num_layers, batch_size, num_heads, seq_len, _ = all_attentions.shape
combined = all_attentions.permute(1, 0, 2, 3, 4)
combined = combined.view(batch_size, num_layers*num_heads, seq_len, seq_len)
for batch in range(batch_size):
    print(f"Batch {batch}: after view, shape {combined[:, batch, :, :].shape}")
    print(combined[:, batch, :, :] )

for batch in range(batch_size):
    print(f"Batch {batch} after permute")
    print(combined[batch, :, :, :] )


all_attentions shape: torch.Size([2, 2, 3, 2, 2])
tensor([[[[[-1, -1],
           [-1, -1]],

          [[-2, -2],
           [-2, -2]],

          [[-3, -3],
           [-3, -3]]],


         [[[ 1,  1],
           [ 1,  1]],

          [[ 2,  2],
           [ 2,  2]],

          [[ 3,  3],
           [ 3,  3]]]],



        [[[[-4, -4],
           [-4, -4]],

          [[-5, -5],
           [-5, -5]],

          [[-6, -6],
           [-6, -6]]],


         [[[ 4,  4],
           [ 4,  4]],

          [[ 5,  5],
           [ 5,  5]],

          [[ 6,  6],
           [ 6,  6]]]]])
Batch 0 before view
tensor([[[[-1, -1],
          [-1, -1]],

         [[-2, -2],
          [-2, -2]],

         [[-3, -3],
          [-3, -3]]],


        [[[-4, -4],
          [-4, -4]],

         [[-5, -5],
          [-5, -5]],

         [[-6, -6],
          [-6, -6]]]])
Batch 1 before view
tensor([[[[1, 1],
          [1, 1]],

         [[2, 2],
          [2, 2]],

         [[3, 3],
          [3, 3]]],


 

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [24]:
import torch

batch_size = 2
num_layers = 2
num_heads = 3
seq_len = 2

# Создаём список attention maps, по одному для каждого слоя
attentions = []
for layer in range(num_layers):
    layer_heads = []
    for head in range(num_heads):
        value = layer*num_heads + head+1  # уникальное значение для каждого слоя/головы
        # attention shape: (batch_size, seq_len, seq_len)
        att = torch.full((seq_len, seq_len), value)
        layer_heads.append(att)

    layer_tensor = torch.stack(layer_heads, dim=0)
    batch_tensor = torch.stack([layer_tensor*(-1), layer_tensor], dim=0)
    attentions.append(batch_tensor)

In [57]:
max_len = 10
attentions = torch.arange(10*2*max_len*max_len).reshape(10,2,max_len,max_len)
masks = torch.zeros((10, max_len))
for i in range(10):
    masks[i,:i] = 1

mask_2d = masks.unsqueeze(1).unsqueeze(-1) * masks.unsqueeze(1).unsqueeze(-2)

attentions * mask_2d

tensor([[[[   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          ...,
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.]],

         [[   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          ...,
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.]]],


        [[[ 200.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
          ...,
          [   0.,    0.,    0.,  ...,    0.,    0

In [58]:
masks

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]])

In [60]:
mask_2d = masks.unsqueeze(1).unsqueeze(-1) * masks.unsqueeze(1).unsqueeze(-2)

In [61]:
masked_attentions = attentions*mask_2d

In [62]:
masked_attentions.shape

torch.Size([10, 2, 10, 10])

In [71]:
masked_attentions[:5,1,:5,:5]

tensor([[[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[300.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[500., 501.,   0.,   0.,   0.],
         [510., 511.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[700., 701., 702.,   0.,   0.],
         [710., 711., 712.,   0.,   0.],
         [720., 721., 722.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[900., 901., 902., 903.,   0.],
         [910., 911., 912., 913.,   0.],
         [920., 921., 922., 923.,   0.],
         [930., 931., 932., 933.,   0.],
        

In [64]:
symmetric_attentions = (
            masked_attentions + masked_attentions.transpose(-1, -2)
        ) / 2

In [70]:
symmetric_attentions[:5,1,:5,:5,]

tensor([[[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000]],

        [[300.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000]],

        [[500.0000, 505.5000,   0.0000,   0.0000,   0.0000],
         [505.5000, 511.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000]],

        [[700.0000, 705.5000, 711.0000,   0.0000,   0.0000],
         [705.5000

In [75]:
300 - 300*300/300

0.0

In [97]:
(505.5+505.5+500+511)

2022.0

In [98]:
505.5 - ((1005.5*1016.5) / (505.5+505.5+500+511))

0.014960435212685752

In [72]:
def apply_apc_correction( attention_maps):
    """
    Apply Average Product Correction (APC) tensorized version.
    F^APC_ij = F_ij - (F_i * F_j) / F
    """
    # attention_maps: (batch_size, num_maps, seq_len, seq_len)

    # Compute row and column sums
    F_i = attention_maps.sum(dim=-1, keepdim=True)  # (batch, num_maps, seq_len, 1)
    F_j = attention_maps.sum(dim=-2, keepdim=True)  # (batch, num_maps, 1, seq_len)
    F_total = attention_maps.sum(
        dim=(-2, -1), keepdim=True
    )  # (batch, num_maps, 1, 1)

    # Apply APC correction
    F_apc = attention_maps - (F_i * F_j) / (F_total + 1e-8)

    return F_apc

In [80]:
F_i = symmetric_attentions.sum(dim=-1, keepdim=True)  # (batch, num_maps, seq_len, 1)
F_j = symmetric_attentions.sum(dim=-2, keepdim=True)  # (batch, num_maps, 1, seq_len)
F_total = symmetric_attentions.sum(
    dim=(-2, -1), keepdim=True
)  # (batch, num_maps, 1, 1)

# Apply APC correction
F_apc = symmetric_attentions - (F_i * F_j) / (F_total + 1e-8)


In [94]:
F_total[:5,1,:,:]

tensor([[[    0.]],

        [[  300.]],

        [[ 2022.]],

        [[ 6399.]],

        [[14664.]]])

In [85]:
F_i[:5, 1, :5, :]

tensor([[[   0.0000],
         [   0.0000],
         [   0.0000],
         [   0.0000],
         [   0.0000]],

        [[ 300.0000],
         [   0.0000],
         [   0.0000],
         [   0.0000],
         [   0.0000]],

        [[1005.5000],
         [1016.5000],
         [   0.0000],
         [   0.0000],
         [   0.0000]],

        [[2116.5000],
         [2133.0000],
         [2149.5000],
         [   0.0000],
         [   0.0000]],

        [[3633.0000],
         [3655.0000],
         [3677.0000],
         [3699.0000],
         [   0.0000]]])

In [86]:
F_j[:5, 1, : ,:5]

tensor([[[   0.0000,    0.0000,    0.0000,    0.0000,    0.0000]],

        [[ 300.0000,    0.0000,    0.0000,    0.0000,    0.0000]],

        [[1005.5000, 1016.5000,    0.0000,    0.0000,    0.0000]],

        [[2116.5000, 2133.0000, 2149.5000,    0.0000,    0.0000]],

        [[3633.0000, 3655.0000, 3677.0000, 3699.0000,    0.0000]]])

In [91]:
1005.5*1016.5

1022090.75

In [92]:
1016.5*1016.5

1033272.25

In [88]:
(F_i * F_j)[:5, 1, :5, :5]

tensor([[[       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000]],

        [[   90000.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000],
         [       0.0000,        0.0000,        0.0000,        0.0000,
                 0.0000]],

        [[ 1011030.2500,  1022090.75

In [73]:
apc_corrected = apply_apc_correction(symmetric_attentions)

In [2]:
import torch

a = torch.arange(20)
print(a)

b = a.reshape(2,2, 5)
print(b)

c = b.transpose(0, 1)
print(c)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])
tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9]],

        [[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]])
tensor([[[ 0,  1,  2,  3,  4],
         [10, 11, 12, 13, 14]],

        [[ 5,  6,  7,  8,  9],
         [15, 16, 17, 18, 19]]])


In [12]:
a = torch.arange(20).reshape(2,-1,2)

In [16]:
a.squeeze(-1).shape

torch.Size([2, 5, 2])

In [17]:
a.shape

torch.Size([2, 5, 2])

In [6]:
b.permute(2,1,0)[:,1,0]

tensor([5, 6, 7, 8, 9])

In [18]:
a = torch.arange(10)
print(a)
a[1:-1]

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [59]:
a = torch.arange(1000,dtype=torch.float32)
a = a.reshape(-1, 10,10)

In [66]:
pos_i = torch.arange(10).unsqueeze(1)
pos_j = torch.arange(10).unsqueeze(0)
separation = (pos_i - pos_j) >= 6
separation

tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False]])

In [73]:
res = (a*separation).view(a.shape[0],-1)

In [80]:
res.shape

torch.Size([10, 100])

In [85]:
torch.argsort(res, dim=-1, stable=True, descending=True)[:,:5]

tensor([[93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82],
        [93, 92, 91, 90, 82]])

In [87]:
a

tensor([[[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.],
         [ 10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.],
         [ 20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.],
         [ 30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
         [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.],
         [ 50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.],
         [ 60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.],
         [ 70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
         [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.],
         [ 90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.]],

        [[100., 101., 102., 103., 104., 105., 106., 107., 108., 109.],
         [110., 111., 112., 113., 114., 115., 116., 117., 118., 119.],
         [120., 121., 122., 123., 124., 125., 126., 127., 128., 129.],
         [130., 131., 132., 133., 134., 135., 136., 137., 138., 139.],
    

In [98]:
import torch

def precision_cutoff(true_labels, predictions, k):
    sorted_indices = torch.argsort(predictions, stable=True, descending=True)
    sorted_indices = sorted_indices[:k]  # Fixed typo in variable name 
    sorted_true_labels = true_labels[sorted_indices]
    print(sorted_true_labels)
    return sorted_true_labels.sum() / k

# Test the function
true_labels = torch.tensor([0, 0, 1, 0, 1])
predictions = torch.tensor([0.9, 0.1, 0.8, 0.3, 0.7])
k = 3

precision = precision_cutoff(true_labels, predictions, k)
print(f"Precision@{k}: {precision}")







tensor([0, 1, 1])
Precision@3: 0.6666666865348816


In [101]:
import torcheval

In [16]:
from lightning.pytorch.loggers import WandbLogger
import lightning as L

logger = WandbLogger(
    project="deep-origin-task",
    name="test",
    log_model=False,
    config = {'test': 'test'}
)
logger.experiment.config.update({})

In [12]:
trainer = L.Trainer(logger=[logger])

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [21]:
logger.log_metrics({"loss": 1})

In [27]:
logger.experiment.name

'test'